# **Love Song Challenge**

This challenge uses RNN (Recurrent Neural Networks) to generate a love song. The RNN is trained using a .txt file containing some Love song lyrics and -given an initial word or a sequence of characters- generates a love song.

![Image from the Challenge annoucement in Slack](https://files.slack.com/files-pri/TQEPF835E-FSTUSSRH6/template.png)

In [0]:
## Importing the necessary libraries
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [3]:
# open text file and read in data as `text`
with open("lovelyrics.txt", 'r') as f:
    text = f.read()

print(text[:100])

Doesn't take much to make me happy
And make me smile with glee 
Never never will I feel discouraged 


# Tokenization

In [0]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

We can see the encoded characters :

In [5]:
encoded[:100]

array([49, 69, 22, 32, 53, 42, 93, 37, 93, 86, 50, 22, 37, 40, 12, 26, 59,
       37, 93, 69, 37, 40, 86, 50, 22, 37, 40, 22, 37, 59, 86, 79, 79,  5,
       55, 21, 53, 97, 37, 40, 86, 50, 22, 37, 40, 22, 37, 32, 40, 76, 29,
       22, 37, 85, 76, 93, 59, 37,  4, 29, 22, 22, 37, 55, 28, 22, 52, 22,
       90, 37, 53, 22, 52, 22, 90, 37, 85, 76, 29, 29, 37, 89, 37, 41, 22,
       22, 29, 37, 97, 76, 32, 26, 69, 12, 90, 86,  4, 22, 97, 37])

# Pre-Processing

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

# Making training mini-batches

In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## Get the number of batches we can make
    n_batches = len(arr) // (batch_size * seq_length)
    
    ## Keep only enough characters to make full batches
    arr = arr[:(n_batches * batch_size * seq_length)]
    
    ## Reshape into batch_size rows
    arr = arr.reshape(batch_size, (n_batches * seq_length))
    
    ## Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

# Defining the network

In [8]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class SongRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## define the layers of the model
        self.lstm = nn.LSTM(input_size=len(tokens), hidden_size=self.n_hidden, num_layers=self.n_layers, bias=True, batch_first=True, dropout=self.drop_prob)
        self.fc = nn.Linear(self.n_hidden, len(tokens))
        self.dropout = nn.Dropout(drop_prob)
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## Get the outputs and the new hidden state from the lstm
        r_out, hidden = self.lstm(x, hidden)
        out = self.dropout(r_out)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)  

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        
        return hidden

# Train the RNN

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    min_validation_loss = float('inf')
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                avg_validation_loss = np.mean(val_losses)
                if avg_validation_loss < min_validation_loss:
                    print("min_validation_loss update:", min_validation_loss, "-->", avg_validation_loss)
                    min_validation_loss = avg_validation_loss
                    torch.save(net.state_dict(), 'Love_Song_model.pth')
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(avg_validation_loss))

# Instantiating the model

In [13]:
# define and print the net
n_hidden=512
n_layers=2

net = SongRNN(chars, n_hidden, n_layers)
print(net)

SongRNN(
  (lstm): LSTM(99, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=99, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


# Set the training hyperparameters!

In [14]:
batch_size = 128
seq_length = 100
n_epochs = 30

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

min_validation_loss update: inf --> 3.3208765983581543
Epoch: 1/30... Step: 10... Loss: 3.3307... Val Loss: 3.3209
min_validation_loss update: 3.3208765983581543 --> 3.2115494864327565
Epoch: 1/30... Step: 20... Loss: 3.2202... Val Loss: 3.2115
min_validation_loss update: 3.2115494864327565 --> 3.2022481645856584
Epoch: 1/30... Step: 30... Loss: 3.1878... Val Loss: 3.2022
min_validation_loss update: 3.2022481645856584 --> 3.197141374860491
Epoch: 1/30... Step: 40... Loss: 3.1674... Val Loss: 3.1971
Epoch: 1/30... Step: 50... Loss: 3.1379... Val Loss: 3.1976
Epoch: 1/30... Step: 60... Loss: 3.1586... Val Loss: 3.1974
min_validation_loss update: 3.197141374860491 --> 3.1939898899623325
Epoch: 2/30... Step: 70... Loss: 3.1520... Val Loss: 3.1940
min_validation_loss update: 3.1939898899623325 --> 3.191684212003435
Epoch: 2/30... Step: 80... Loss: 3.1446... Val Loss: 3.1917
min_validation_loss update: 3.191684212003435 --> 3.1891769681658064
Epoch: 2/30... Step: 90... Loss: 3.1344... Val Lo

# Checkpoint

In [0]:
# change the name, for saving multiple files
model_name = 'rnn_love_song.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

# Predictions

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

# Priming and generating text 

In [0]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [26]:
print(sample(net, 1000, prime='You', top_k=3))

Your lovin' you will be
All you need is all I need is you.
I want you to say I'm so into you

There's no mutic to my life with you

That you've got me time and I want to be your lips 

I've got to let you go 
I will not let the will walk
I want to live in love 
I can't let you go
I can't live it all I wanna be with you
I'll be there that you want more 
I don't know how that it things 
I wanna be with you

I'm a little but in me
This mant to made

There are the same of my heart I wanna keep you the sea

If you can't wait to be all that I wanna be a like

You got me wanting away
I want to let you know that I will be

I wanna be your here, baby 
I want you to be my baby

The way that I want you to see you want me
I wanna be with you baby
I will love you for all that I let you

This much words I wanna keep mine

To me what
I can't live a lit and no
I don't stear what this in this, 
I can't let you know you wild
All the way in my life

I can't lose to love you 
I'll be there to love
I'm som